In [ ]:

!conda info

In [ ]:
# This little hack gets me my local config file (i.e. one that's not in PYTHONPATH):
import sys
sys.path.append('/direct/sdcc+u/dcde1000006/sc19-demo/bnl')
from bnl_config_dirty.py import bnl_sdcc_config

In [ ]:
!ls

In [ ]:
!pwd

In [ ]:
# Set up parsl and fetch my config:
import parsl
import os
import time
from parsl.app.app import python_app, bash_app
from parsl.configs.local_threads import config
parsl.load(config)

In [ ]:


@python_app
def hello ():
    import time
    time.sleep(5)
    return 'Hello World!'

app_future = hello()

# Check if the app_future is resolved
print ('Done: %s' % app_future.done())

# Print the result of the app_future. Note: this
# call will block and wait for the future to resolve
print ('Result: %s' % app_future.result())
print ('Done: %s' % app_future.done())

In [ ]:
# THIS IS LOCAL CONFIG

import parsl
import os
import time
from parsl.app.app import python_app, bash_app
from parsl.configs.local_threads import config


@bash_app
def bnl_node_probe(job_dir=None, stdout=None, stderr=None, mock=False):
    
    cmd_line = '''pwd
echo $HOME
/bin/ls $HOME
/bin/df -h
    '''

    if mock:
        return '''tmp_file=$(mktemp);
cat<<EOF > $tmp_file
{}
EOF
cat $tmp_file
        '''.format(cmd_line)
    else:    
        return(cmd_line)    

    
parsl.clear()    
parsl.load(config)
    
try:
    os.remove('nodeprobe.out')
except OsError:
    pass

try:
    os.remove('nodeprobe.err')
except OsError:
    pass

w = bnl_node_probe(stdout='nodeprobe.out', stderr='nodeprobe.err')

print('Sleeping a little now...\n')
time.sleep(5)

w.result()
print('Contents of stdout file follow:\n')
with open(w.stdout, 'r') as f:
    print(f.read())

In [6]:
# THIS WILL BE REMOTE CONFIG (actually we should be able to switch w/ different configs!)


import parsl
import os
import time


from parsl.config import Config

from parsl.channels import OAuthSSHChannel
from parsl.providers import CondorProvider
from parsl.launchers import SrunLauncher
from parsl.executors import HighThroughputExecutor
from parsl.addresses import address_by_hostname

from parsl.app.app import python_app, bash_app
from parsl.configs.local_threads import config



bnl_sdcc_config = Config(
    app_cache=True,
    checkpoint_files=None,
    checkpoint_mode=None,
    checkpoint_period=None,
    data_management_max_threads=10,
    executors=[HighThroughputExecutor(
        #address='127.0.0.1',
        address='130.199.185.13',
        cores_per_worker=1,
        heartbeat_period=30,
        heartbeat_threshold=120,
        #interchange_port_range=(55000, 56000),
        label='spce01.sdcc.bnl.gov-htcondor',
        launch_cmd='process_worker_pool.py {debug} {max_workers} -p {prefetch_capacity} -c {cores_per_worker} -m {mem_per_worker} --poll {poll_period} --task_url={task_url} --result_url={result_url} --logdir={logdir} --block_id={{block_id}} --hb_period={heartbeat_period} --hb_threshold={heartbeat_threshold} ',
        mem_per_worker=4,
        managed=True,
        max_workers=1,
        poll_period=10,
        prefetch_capacity=0,
        provider=CondorProvider(
            channel=OAuthSSHChannel(
                'spce01.sdcc.bnl.gov',
                envs={},
                port=2222,
                script_dir='/sdcc/u/dcde1000006/parsl_scripts',
                username='dcde1000006'
            ),
            environment={},
            init_blocks=1,
            # launcher=SingleNodeLauncher(),
            max_blocks=1,
            min_blocks=1,
            nodes_per_block=1,
            #parallelism=1,
            parallelism=0,
            project='',
            #Trying this Requirements directive per Dong's instructions:
            #requirements='regexp("^sp[oa]", machine)',
            scheduler_options='accounting_group = group_sdcc.main \nRequirements = (regexp("^sp[oa]", machine))',
            transfer_input_files=[],
            walltime='00:10:00',
            #worker_init='source /sdcc/u/dcde1000001/dcdesetup.sh'
            worker_init='source /hpcgpfs01/work/dcde/setup.sh; source activate dcdemaster20191008'
        ),
        storage_access=[],
        suppress_failure=False,
        worker_debug=True,
        worker_logdir_root='/sdcc/u/dcde1000006/parsl_scripts/logs',
        worker_port_range=(50000, 51000),
        #worker_port_range=(5000, 5100),   # per John H's message 8/29/19
        worker_ports=None,
        working_dir='/sdcc/u/dcde1000006/parsl_scripts'
    )],
    lazy_errors=True,
    monitoring=None,
    retries=0,
    run_dir='runinfo',
    strategy='simple',
    usage_tracking=False
)

@python_app
def show_env():
    import os, os.path
    stuff = []

    stuff.append('Working dir is {}'.format(os.getcwd()))
    stuff.append(os.environ['HOME'])
    
    return stuff


@bash_app
def bnl_node_probe(job_dir=None, stdout=None, stderr=None, mock=False):
    
    cmd_line = '''pwd
echo $HOME
/bin/ls $HOME
/bin/df -h
    '''

    if mock:
        return '''tmp_file=$(mktemp);
cat<<EOF > $tmp_file
{}
EOF
cat $tmp_file
        '''.format(cmd_line)
    else:    
        return(cmd_line)    
    
try:
    os.remove('nodeprobe.out')
except OSError:
    pass
except FileNotFoundError:
    pass

try:
    os.remove('nodeprobe.err')
except OSError:
    pass
except FileNotFoundError:
    pass

#parsl.set_stream_logger()
jal
parsl.clear()    
parsl.load(config)
#parsl.load(bnl_sdcc_config)

x = show_env()
if x.done():
  print(format(x.result()))


w = bnl_node_probe(stdout='nodeprobe.out', stderr='nodeprobe.err')

#print('Sleeping a little now...\n')
#time.sleep(5)

""" example from parsl docs:
with open(hello.outputs[0].result().filepath, 'r') as f:
     print(f.read())
"""

#w.result()
if w.done():
    print('Contents of stdout file follow:\n')
    with open(w.stdout, 'r') as f:
        print(f.read())

In [5]:
!ls -aR /hpcgpfs01/scratch/dcde1000006

/hpcgpfs01/scratch/dcde1000006:
.  ..  parsl_jobs  relion-tut

/hpcgpfs01/scratch/dcde1000006/parsl_jobs:
.  ..

/hpcgpfs01/scratch/dcde1000006/relion-tut:
.  ..  relion21_tutorial

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial:
.  ..  betagal	PrecalculatedResults  relion21_tutorial.pdf

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/betagal:
.  ..  Micrographs  mtf_falcon2_300kV.star

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/betagal/Micrographs:
.
..
Falcon_2012_06_12-14_33_35_0_movie.mrcs
Falcon_2012_06_12-14_57_34_0_movie.mrcs
Falcon_2012_06_12-15_14_01_0_movie.mrcs
Falcon_2012_06_12-15_41_22_0_movie.mrcs
Falcon_2012_06_12-15_53_09_0_movie.mrcs
Falcon_2012_06_12-15_56_10_0_movie.mrcs
Falcon_2012_06_12-16_26_22_0_movie.mrcs
Falcon_2012_06_12-16_44_07_0_movie.mrcs
Falcon_2012_06_12-16_55_40_0_movie.mrcs
Falcon_2012_06_12-16_59_12_0_movie.mrcs
Falcon_2012_06_12-17_02_43_0_movie.mrcs
Falcon_2012_06_12-17_14_17_0_movie.mrcs
Falcon_2012_06_12-

.				 run_it012_optimiser.star
..				 run_it012_sampling.star
default_pipeline.star		 run_it013_class001_angdist.bild
job_pipeline.star		 run_it013_class001.mrc
note.txt			 run_it013_class002_angdist.bild
run.err				 run_it013_class002.mrc
.run.err.tail			 run_it013_class003_angdist.bild
run_it000_class001_angdist.bild  run_it013_class003.mrc
run_it000_class001.mrc		 run_it013_class004_angdist.bild
run_it000_class002_angdist.bild  run_it013_class004.mrc
run_it000_class002.mrc		 run_it013_data.star
run_it000_class003_angdist.bild  run_it013_model.star
run_it000_class003.mrc		 run_it013_optimiser.star
run_it000_class004_angdist.bild  run_it013_sampling.star
run_it000_class004.mrc		 run_it014_class001_angdist.bild
run_it000_data.star		 run_it014_class001.mrc
run_it000_model.star		 run_it014_class002_angdist.bild
run_it000_optimiser.star	 run_it014_class002.mrc
run_it000_sampling.star		 run_it014_class003_angdist.bild
run_it001_class001_angdist.bild  run_it014_class003.mrc
run_it001_class0

.		       job_pipeline.star  particles.star  run.job
..		       Micrographs	  run.err	  run.out
default_pipeline.star  note.txt		  .run.err.tail   .run.out.tail

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/Extract/job011/Micrographs:
.
..
Falcon_2012_06_12-14_33_35_0_extract.star
Falcon_2012_06_12-14_33_35_0.mrcs
Falcon_2012_06_12-14_57_34_0_extract.star
Falcon_2012_06_12-14_57_34_0.mrcs
Falcon_2012_06_12-15_14_01_0_extract.star
Falcon_2012_06_12-15_14_01_0.mrcs
Falcon_2012_06_12-15_41_22_0_extract.star
Falcon_2012_06_12-15_41_22_0.mrcs
Falcon_2012_06_12-15_53_09_0_extract.star
Falcon_2012_06_12-15_53_09_0.mrcs
Falcon_2012_06_12-15_56_10_0_extract.star
Falcon_2012_06_12-15_56_10_0.mrcs
Falcon_2012_06_12-16_26_22_0_extract.star
Falcon_2012_06_12-16_26_22_0.mrcs
Falcon_2012_06_12-16_44_07_0_extract.star
Falcon_2012_06_12-16_44_07_0.mrcs
Falcon_2012_06_12-16_55_40_0_extract.star
Falcon_2012_06_12-16_55_40_0.mrcs
Falcon_2012_06_12-16_59_12_0_extract.sta

.  ..  run_half1_class001_unfil.mrc

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/11:
.  ..  LocalRes  Polish  PostProcess

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/11/LocalRes:
.  ..  first3dref

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/11/LocalRes/first3dref:
.  ..  relion_locres_filtered.mrc

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/11/Polish:
.  ..  first3dref

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/11/Polish/first3dref:
.  ..  shiny_post.mrc

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/11/PostProcess:
.  ..  first3dref  first3dref_fliphand	shiny

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/11/PostProcess/first3dref:
.  ..  postprocess_masked.mrc  postprocess.mrc

/h


/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/7/MaskCreate/job021:
.  ..  mask.mrc

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/8:
.  ..  Class2D	Class3D  InitialModel

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/8/Class2D:
.  ..  after_sorting  manualpick

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/8/Class2D/after_sorting:
.  ..  run_it025_model.star

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/8/Class2D/manualpick:
.  ..  run_it025_model.star

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/8/Class3D:
.  ..  first_exhaustive

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/.Nodes/8/Class3D/first_exhaustive:
.  ..  run_it025_model.star

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/P

.				       run_it006_half2_class001.mrc
..				       run_it006_half2_model.star
default_pipeline.star		       run_it006_optimiser.star
job_pipeline.star		       run_it006_sampling.star
note.txt			       run_it007_data.star
run_class001_angdist.bild	       run_it007_half1_class001_angdist.bild
run_class001.mrc		       run_it007_half1_class001.mrc
run_data.star			       run_it007_half1_model.star
run.err				       run_it007_half2_class001_angdist.bild
.run.err.tail			       run_it007_half2_class001.mrc
run_fliphand_half1_class001_unfil.mrc  run_it007_half2_model.star
run_fliphand_half2_class001_unfil.mrc  run_it007_optimiser.star
run_half1_class001_unfil.mrc	       run_it007_sampling.star
run_half2_class001_unfil.mrc	       run_it008_data.star
run_it000_data.star		       run_it008_half1_class001_angdist.bild
run_it000_half1_class001_angdist.bild  run_it008_half1_class001.mrc
run_it000_half1_class001.mrc	       run_it008_half1_model.star
run_it000_half1_model.star	       run_it008_half2_

.		       job_pipeline.star	  run.err	 run.out
..		       micrographs_selected.star  .run.err.tail  .run.out.tail
default_pipeline.star  note.txt			  run.job

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/Select/job013:
.		       job_pipeline.star  .run.err.tail  .run.out.tail
..		       note.txt		  run.job
backup_selection.star  particles.star	  run.out
default_pipeline.star  run.err		  run.out~

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/Select/job015:
.		       class_averages.star    particles.star  run.out
..		       default_pipeline.star  run.err	      .run.out.tail
backup_selection.star  job_pipeline.star      .run.err.tail
class_averages.mrcs    note.txt		      run.job

/hpcgpfs01/scratch/dcde1000006/relion-tut/relion21_tutorial/PrecalculatedResults/Select/job019:
.		       default_pipeline.star  particles.star  run.job
..		       job_pipeline.star      run.err	      run.out
backup_selection.star  note.txt	